In [ ]:
!pip install -q hashstash[rec] stanza

In [ ]:
import stanza
stanza.Pipeline()

In [ ]:
NLP = None
def get_nlp():
    global NLP
    if NLP is None:
        NLP = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,ner,depparse,constituency', verbose=0)
    return NLP

In [ ]:
# !unzip -n osp_slices_1000.zip
# !mkdir -p /root/.cache/hashstash
# !rm -rf /root/.cache/hashstash/osp_slices_1000
# !mv osp_slices_1000 /root/.cache/hashstash

In [ ]:
from hashstash import HashStash


input_stash = HashStash('osp_slices_1000')
input_stash

Config,Param,Value
Path,Root Dir,/root/.cache/hashstash/osp_slices_1000
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,57620


In [ ]:
import string
def remove_left_right_punct(text):
    """
    Removes leading and trailing punctuation from a string.
    """
    return text.strip(string.punctuation)


In [ ]:
output_stash = HashStash('osp_slices_1000_nlp')
output_stash


In [ ]:
text_id,slice_d = next(input_stash.items())
slice_id, slice_txt = list(slice_d.items())[0]
# slice_id, slice_txt

In [ ]:
def gen_nlp_doc(txt, key, force=False, stash=output_stash):
    if force or key not in stash:
        nlp = get_nlp()
        doc = nlp(txt)
        stash[key] = doc.to_serialized()
        return doc

In [ ]:
# gen_nlp_doc(slice_txt, f'{text_id}__{int(slice_id):02d}')

In [ ]:
from tqdm import tqdm

In [ ]:
inps = [(f'{text_id}__{int(slice_id):02d}', _) for text_id, slice_d in tqdm(input_stash.items(),total=len(input_stash)) for slice_id, _ in slice_d.items()]
inps[0]

100%|██████████| 57620/57620 [00:16<00:00, 3412.85it/s]


('phil/10.2307/45238188__01', 'My argument has three steps.\nFirst, I argue that proponents of conciliatory policies have good reason to affirm a view that I call "instrumentalism," a view that commends treating our doxastic inclinations like instrumental readouts.\nSecond, I show that instrumentalism supplies a basis for demanding conciliatory requirements in superficial disagreements but not in fundamental disagreements.\nThird, I argue that the frequently invoked "independence" principle, which arguably would require significant conciliation in fundamental disputes, is unmotivated in light of the explanatory power of instrumentalism.\nThe most plausible conciliatory view, then, is a weak conciliationism that features instrumentalism rather than independence as the central principle, and that therefore gives us a principled basis for thinking that fundamental disagreements should occasion less doxastic revision than shallow disagreements.\nIn the course of developing this explanation

In [722]:
import random
random.shuffle(inps)

In [723]:
for key, txt in tqdm(inps):
    gen_nlp_doc(txt, key)

  0%|          | 269/186708 [07:38<88:17:57,  1.70s/it]


KeyboardInterrupt: 

In [ ]:
# !zip -r -q osp_slices_1000_nlp.zip /root/.cache/hashstash/osp_slices_1000_nlp

In [ ]:
key,docstr = next(output_stash.items())
doc = stanza.Document.from_serialized(docstr)
word = doc.sentences[1].words[5]
key

'phil/10.2307/20140633__02'

In [ ]:
def get_word_context(doc, sent_i, word_i, context_len=2):
    sent = doc.sentences[sent_i]

    prev_context=''
    next_context=''

    words_forward=sent.words[word_i+1:]
    words_backward=reversed(sent.words[:word_i])
    for w in words_forward:
        if len(next_context) < context_len:
            next_context+=w.text+' '
        else:
            break
    for w in words_backward:
        if len(prev_context) < context_len:
            prev_context=w.text+' '+prev_context+' '
        else:
            break

    word = sent.words[word_i]
    out = f'{prev_context.strip()} {word.text.upper()} {next_context.strip()}'
    out = out.replace('\n',' ').replace(' ,',',').replace(' .', '.').replace(' !','!').replace(' ?','?')
    out = out.replace('( ','(').replace('[ ','[').strip().replace(' )',')').replace(' ]',']')
    return remove_left_right_punct(out).strip()

In [ ]:
# get_word_context(doc, 2, 12)

In [ ]:
# def get_token_eg(doc,word):
#     return

In [740]:


def get_pos_counts(doc, feat2word2count=None, feat2word2eg=None):
    pos_counts = Counter()
    deprel_counts = Counter()
    for sent_i,sent in enumerate(doc.sentences):
        for word_i,word in enumerate(sent.words):
            pos_counts[word.pos]+=1
            deprel_counts[word.deprel]+=1

            eg_word = word.text.lower()
            if feat2word2count is not None:
                feat2word2count[word.deprel][eg_word]+=1
                feat2word2count[word.pos][eg_word]+=1

            if feat2word2eg is not None:
                eg_context = get_word_context(doc, sent_i, word_i, context_len=context_len).strip()
                feat2word2eg[word.deprel][eg_word] = eg_context
                feat2word2eg[word.pos][eg_word] = eg_context

    sum_pos_counts = sum(pos_counts.values())
    # print(sum_pos_counts)
    pos_counts_rel = {k:int(round(v/sum_pos_counts*1000)) for k,v in pos_counts.items()}

    sum_deprel_counts = sum(deprel_counts.values())
    deprel_counts_rel = {k:int(round(v/sum_deprel_counts*1000)) for k,v in deprel_counts.items()}

    return {**pos_counts_rel,**deprel_counts_rel}



In [741]:


from collections import Counter, defaultdict
feat2word2count = defaultdict(Counter)
disc2feat2word2count = {
    'phil': defaultdict(Counter),
    'lit': defaultdict(Counter)
}

disc2feat2word2eg={
    'phil': defaultdict(dict),
    'lit': defaultdict(dict)
}

context_len = 15


id2counts = {}

for id,docstr in tqdm(output_stash.items(),total=len(output_stash)):
    if id not in id2counts:
        doc = stanza.Document.from_serialized(docstr)
        disc = id.split('/')[0]
        feat2word2count = disc2feat2word2count[disc]
        feat2word2eg = disc2feat2word2eg[disc]
        id2counts[id] = get_pos_counts(doc, feat2word2count=feat2word2count, feat2word2eg=feat2word2eg)
    if len(id2counts) > 1000:
        break

 36%|███▌      | 1000/2773 [01:07<01:59, 14.85it/s]


In [742]:
# get_pos_counts(doc)

In [745]:
FEAT_N = 100
FEAT_MIN_COUNT = 0

def get_egs(word2count, n = FEAT_N, min_count=FEAT_MIN_COUNT, word2eg={}):
    total = word2count.total()
    o = []
    for w,c in word2count.most_common():
        if n and len(o) >= n:
            break

        c = int(round(c/total*1000))
        if c >= min_count:
            # o.append(f'{w} ({c})')
            eg = word2eg.get(w)
            if eg:
                o.append(f'{w.upper()} ({c}) ["{eg}"]')
            else:
                o.append(f'{w} ({c})')
            n-=1
        else:
            break
        # o.append(w)
    return '; '.join(o)

def get_egs_feat(feat, disc='phil', n=FEAT_N, min_count=FEAT_MIN_COUNT, incl_egs=False):
    feat2word2count = disc2feat2word2count[disc]
    feat2word2eg = disc2feat2word2eg[disc]
    word2count = feat2word2count[feat]
    word2eg = feat2word2eg[feat]
    return get_egs(word2count, n=n, min_count=min_count, word2eg=word2eg if incl_egs else {})


In [746]:
# get_egs_feat('nsubj', disc='phil')

In [747]:
# get_egs_feat('amod', disc='lit',incl_egs=True,n=20)

In [749]:
import pandas as pd

pd.options.display.max_colwidth=None

df = pd.DataFrame(id2counts).T.rename_axis('id')
df['_target'] = [i.split('/')[0] for i in df.index]
# df.columns = [f'{x} ({get_egs(feat2word2count[x])})' for x in df]
df

,PRON,AUX,VERB,CCONJ,ADP,PUNCT,NOUN,ADV,PROPN,PART,...,vocative,advcl:relcl,nmod:unmarked,iobj,csubj:outer,dislocated,csubj:pass,orphan,goeswith,_target
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/20140633__02,88.0,66.0,99.0,34.0,115.0,98.0,189.0,47.0,25.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil
phil/10.2307/43921200__10,7.0,6.0,16.0,42.0,78.0,234.0,204.0,6.0,258.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil
phil/10.2307/26706722__04,38.0,76.0,78.0,21.0,110.0,121.0,215.0,64.0,18.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil
phil/10.2307/2180906__06,46.0,51.0,88.0,30.0,134.0,137.0,198.0,44.0,44.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phil
lit/469190__01,62.0,59.0,82.0,38.0,111.0,132.0,210.0,35.0,8.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
phil/10.2307/42971411__08,45.0,68.0,93.0,34.0,95.0,137.0,239.0,34.0,9.0,27.0,...,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,phil
lit/510130__02,58.0,59.0,95.0,28.0,130.0,133.0,173.0,47.0,49.0,28.0,...,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,lit
lit/3713731__02,72.0,63.0,103.0,33.0,104.0,126.0,135.0,27.0,100.0,29.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lit


In [750]:
df_means = df.groupby('_target').mean()
feat2grp2mean = df_means.to_dict()

ld = []
feat1 = None
for feat in feat2grp2mean:

    feat_d = {'feat':feat}
    grp2mean = feat2grp2mean[feat]
    grp1 = None
    for grp,score in grp2mean.items():
        feat_d[grp+'_egs'] = get_egs_feat(feat, disc=grp, min_count=0, n=25, incl_egs=False)
        feat_d[grp+'_egs2'] = get_egs_feat(feat, disc=grp, n=25, incl_egs=True)

        feat_d[grp]=score

    grps = list(grp2mean.keys())
    if len(grps)>1:
        grp1=grps[0]
        for grp2 in grps[1:]:
            feat_d[f'{grp2}-{grp1}'] = feat_d[grp2] - feat_d[grp1]

    # feat1 = feat
    ld.append(feat_d)
odf = pd.DataFrame(ld).dropna().sort_values('phil-lit',ascending=False).set_index('feat')
for c in odf:
    if not '_egs' in c:
        odf[c] = odf[c].apply(round).apply(int).astype(int)
odf = odf[['phil','lit','phil-lit','phil_egs','lit_egs','phil_egs2','lit_egs2']]

In [753]:
# @title Most philosophy
odf.sort_values('phil-lit', ascending=False).head(10)

,phil,lit,phil-lit,phil_egs,lit_egs,phil_egs2,lit_egs2
feat,,,,,,,
AUX,65,47,18,is (323); be (139); are (104); can (71); would (36); will (34); may (30); does (30); have (27); must (21); has (20); was (19); being (18),is (291); be (99); are (80); was (75); have (50); can (40); has (37); been (33); would (33); may (30); were (29); will (26); had (24),"IS (323) [""It IS of course being""]; BE (139) [""it may still BE true that different""]; ARE (104) [""A few facts ARE nevertheless known""]; CAN (71) [""We CAN distinguish them""]; WOULD (36) [""ghetto problems WOULD be solved if ghetto""]; WILL (34) [""a septic tank WILL not dry up, collect""]; MAY (30) [""For it MAY still be true that""]; DOES (30) [""pronominalization DOES not cause any special""]; HAVE (27) [""I HAVE located only two""]; MUST (21) [""Instead, we MUST allow branching""]; HAS (20) [""every critic HAS reviewed at least""]; WAS (19) [""if the message WAS completely destroyed""]; BEING (18) [""is of course BEING assumed here that""]","IS (291) [""final effect IS coalescence""]; BE (99) [""He might also BE taxed with setting""]; ARE (80) [""and the Fool ARE the sublimest instance""]; WAS (75) [""He WAS perhaps rashly""]; HAVE (50) [""the longer they HAVE been kept asunder""]; CAN (40) [""Espronceda CAN scarcely have had""]; HAS (37) [""Shakespeare or Milton HAS achieved "" a deeper""]; BEEN (33) [""longer they have BEEN kept asunder""]; WOULD (33) [""characters, WOULD be false and sentimental""]; MAY (30) [""incongruities MAY lend even to tragic""]; WERE (29) [""Verdugo, which WERE duly published""]; WILL (26) [""professional satirist WILL naturally insinuate""]; HAD (24) [""that Hazlitt HAD analyzed in more""]"
NOUN,196,180,16,theory (8); case (7); sense (7); way (7); fact (6); truth (6); knowledge (5); view (5); world (5); question (5); argument (5); belief (4); time (4),life (6); time (6); world (5); way (5); man (4); work (4); sense (4); play (4); language (4); self (4); form (4); fact (4); history (3),"THEORY (8) [""known as the THEORY of finite partially""]; CASE (7) [""not always the CASE in real games""]; SENSE (7) [""latter in the SENSE that the move connected""]; WAY (7) [""obtainable in this WAY, with the obvious""]; FACT (6) [""It is in FACT easy to find any""]; TRUTH (6) [""there is one TRUTH, which is supposed""]; KNOWLEDGE (5) [""have detailed KNOWLEDGE of anything specific""]; VIEW (5) [""theoretical point of VIEW""]; WORLD (5) [""considering a WORLD co and a sentence""]; QUESTION (5) [""It is a QUESTION of the locus of""]; ARGUMENT (5) [""This ARGUMENT does not consider""]; BELIEF (4) [""is that its BELIEF in tigers provide""]; TIME (4) [""in point of TIME, first comes to""]","LIFE (6) [""observation on human LIFE, without elevation""]; TIME (6) [""at the same TIME bring out "" with""]; WORLD (5) [""result is a WORLD of "" seeming anal""]; WAY (5) [""kind, in one WAY or another, limits""]; MAN (4) [""substitution of a MAN for a doomed friend""]; WORK (4) [""In this WORK "" the scenes""]; SENSE (4) [""achieved "" a deeper SENSE than others of""]; PLAY (4) [""As the PLAY opens Dick Dudgeon""]; LANGUAGE (4) [""Virgil in any LANGUAGE""]; SELF (4) [""to save the SELF - sacrificing friend""]; FORM (4) [""in abstract FORM) the climax of""]; FACT (4) [""In FACT, the evident differences""]; HISTORY (3) [""with the later HISTORY of some of the""]"
mark,46,32,14,to (332); that (264); if (80); as (68); of (39); in (28); for (25); because (22); by (22); whether (19); since (14); while (10); though (10),to (381); that (204); as (95); if (47); of (39); in (30); by (23); for (23); because (21); while (16); though (15); than (11); although (10),"TO (332) [""writer they hat TO pronounce themselves""]; THAT (264) [""still be true THAT different writers""]; IF (80) [""However, IF the requirement""]; AS (68) [""degrees, as long AS for each separate""]; OF (39) [""interesting possibility OF characterizing""]; IN (28) [""specialization consists IN maintaining a contextua

In [754]:
# @title Most lit
odf.sort_values('phil-lit', ascending=True).head(25)

,phil,lit,phil-lit,phil_egs,lit_egs,phil_egs2,lit_egs2
feat,,,,,,,
PROPN,21,53,-33,s (13); kant (12); god (10); f (10); aristotle (9); c (8); t (8); a (8); john (7); hume (6); g (6); frege (6); husserl (6),de (10); shakespeare (5); mr. (5); god (4); la (4); england (4); john (3); james (3); king (3); english (3); london (3); m. (3); spenser (3),"S (13) [""semantic question (S) is what Dretske""]; KANT (12) [""KANT 's dictum affirming""]; GOD (10) [""the laws of GOD, and so forth""]; F (10) [""functional question (F""]; ARISTOTLE (9) [""which reinforces ARISTOTLE 's remark that""]; C (8) [""the final C major chord a part""]; T (8) [""D, P, T and R""]; A (8) [""the fact that A has the concept""]; JOHN (7) [""wrinkling of JOHN 's brow is a part""]; HUME (6) [""In this sense HUME is right when he""]; G (6) [""deduced, only G — del sentences""]; FREGE (6) [""Fregean mindful of FREGE 's reflections""]; HUSSERL (6) [""influenced by HUSSERL 's Logical Investigations""]","DE (10) [""Yenes, Calle DE Segovia, Nim""]; SHAKESPEARE (5) [""SHAKESPEARE 's images are still""]; MR. (5) [""Fielding 's play MR. Modern twice suggests""]; GOD (4) [""the theme of GOD 's relation to""]; LA (4) [""En LA Imprenta de Yenes""]; ENGLAND (4) [""been sweeping ENGLAND""]; JOHN (3) [""epigraph from St. JOHN of the Cross""]; JAMES (3) [""sometime neighbor JAMES H. Sledd that our""]; KING (3) [""Lying Valet. KING Henry VIII - Ditto""]; ENGLISH (3) [""arguing that the ENGLISH are more free than""]; LONDON (3) [""that envoys from LONDON visited Teresa""]; M. (3) [""p. ; John M. Headley, e Problem""]; SPENSER (3) [""observation and SPENSER 's fable""]"
X,7,38,-31,de (47); en (31); van (28); een (21); het (20); s (17); et (16); in (13); dat (13); die (13); zijn (9); te (9); met (8),de (35); et (32); la (21); le (16); a (13); que (11); les (10); en (10); des (9); il (9); qui (8); ne (7); un (6),"DE (47) [""la psychologie DE la forme, "" serves""]; EN (31) [""gist et dort, EN ce sollier Qu""]; VAN (28) [""het systeem VAN Kant nauwkeurig""]; EEN (21) [""samenvatting van EEN zeker complex van""]; HET (20) [""vergelijken met HET zoo wonderlijk""]; S (17) [""S or T or Y S, T and Y are compatible""]; ET (16) [""Finally, Dawid ET al""]; IN (13) [""categories at IN de ente, q. Specifically""]; DAT (13) [""en gevolgen, DAT zeer eigenaardige""]; DIE (13) [""Einführung in DIE symbolische Logik""]; ZIJN (9) [""het van belang ZIJN, het systeem van""]; TE (9) [""Kant nauwkeurig TE vergelijken met""]; MET (8) [""te vergelijken MET het zoo wonderlijk""]","DE (35) [""Abrasados en sed DE venganza, Odio""]; ET (32) [""plurimum quoque jure ET merito tribuit""]; LA (21) [""guerreros, De LA patria sosten y""]; LE (16) [""Dans LE troisisme il traite""]; A (13) [""a questi et A quelli""]; QUE (11) [""n'est proprement QUE la pratique de""]; LES (10) [""tels qu'il LES voit), relying""]; EN (10) [""Abrasados EN sed de venganza""]; DES (9) [""laquelle il don ne DES details tres instructifs""]; IL (9) [""sur laquelle IL don ne des details""]; QUI (8) [""negotium, quippe QUI praefecto oratori""]; NE (7) [""laquelle il don NE des details tres""]; UN (6) [""quasi nate ad UN parto (vol""]"
flat,6,27,-21,de (31); van (24); en (18); een (15); het (14); in (10); dat (8); die (8); zijn (8); j. (7); e (7); te (7); met (6),de (41); la (19); le (12); a (11); et (10); que (8); les (8); des (7); en (7); il (6); qui (6); ne (5); e (5),"DE (31) [""la psychologie DE la forme, "" serves""]; VAN (24) [""As Tim VAN Gelder and Robert""]; EN (18) [""gist et dort, EN ce sollier Qu""]; EEN (15) [""samenvatting van EEN zeker complex van""]; HET (14) [""vergelijken met HET zoo wonderlijk""]; IN (10) [""intellectum est IN intelligente immaterialiter""]; DAT (8) [""en gevolgen, DAT zeer eigenaardige""]; DIE (8) [""Hegel "" erfasst DIE Arbeit als das""]; ZIJN (8) [""het van belang ZIJN, het systeem van""]; J. (7) [""OF PROHIBITION J. ELLIOT ROSS""]; E (7) [""of T evidence E is produced in""]; TE (7) [""Kant nauwkeurig TE 